In [1]:
import os
from openai import OpenAI

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 通过 messages 数组实现上下文管理
messages = [
    {'role': 'user', 'content': '你好'}
]

completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)

print("="*20+"第一轮对话"+"="*20)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

messages.append({'role': 'assistant', 'content': completion.choices[0].message.content})
messages.append({'role': 'user', 'content': '你是谁'})
print("="*20+"第二轮对话"+"="*20)
completion = client.chat.completions.create(
    model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称。
    messages=messages
)
# 通过reasoning_content字段打印思考过程
print("="*20+"思考过程"+"="*20)
print(completion.choices[0].message.reasoning_content)
# 通过content字段打印最终答案
print("="*20+"最终答案"+"="*20)
print(completion.choices[0].message.content)

====================第一轮对话====================
====================思考过程====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================最终答案====================
你好！很高兴见到你，有什么我可以帮忙的吗？无论是学习、工作还是生活中的问题，都可以告诉我哦！😊
====================第二轮对话====================
====================思考过程====================
好的，用户现在问“你是谁”，看起来他们想更了解我的身份和功能。之前用户先用中文打招呼，我也用中文回复了，然后他们继续用中文提问，所以整个对话都是中文的。

首先，我需要明确用户的需求。用户可能已经知道我是AI助手，但想了解更详细的信息，比如我的开发团队、功能、数据隐私政策等。也有可能他们是在测试我的回答能力，或者对AI技术本身感兴趣。

接下来，我要考虑之前的对话历史。用户第一次打招呼“你好”，我回应了，然后他们直接问“你是谁”。这说明用户可能在初次接触后，想确认我的身份和可信度。这时候需要提供一个全面但简洁的回答，涵盖我的基本信息和能力，同时让用户感到放心。

然后，我需要确保回答准确无误，符合公司的指导方针。要提到我是由中国的深度求索（DeepSeek）公司开发的智能助手，强调多领域的帮助能力，比如回答问题、提供建议、学习辅助等。同时，必须提及数据隐私和安全性，这是用户可能关心的点。

另外，用户可能希望知道我的局限性和更新时间，这样他们可以合理预期我的回答范围。比如，我的知识截止到2023年12月，无法访问实时信息，这些都需要明确说明，避免误解。

还要保持友好和开放的态度，鼓励用户继续提问，让他们感到被欢迎和支持。使用表情符号可以增加亲切感，但根据用户的风格调整，如果之前用户用了表情，可以适当回应。

最后，检查回答是否符合中文表达习惯，避免技术术语过多，确保清晰易懂。同时，结构要条理分明，分点说明，方便用户快速获取信息。

总结来说，回答需要涵盖身份、功能、隐私、局限性，以及鼓励互动，所有内容都要准确、友好且易于理解。
=

In [16]:
import re

def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名X": "xxx", "特征": "xxx"
例子：
  {{"角色名1": "瑞奇",
  "特征": "25岁黑色短发的男性",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}
注意：
1. 角色名应当真实在文本中出现。
2. 特征包括年龄、外貌、衣着、行为、情绪等明显描述。
3. 不需要编造额外信息。

小说内容如下：
{text}
"""
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key="sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    
    response = client.chat.completions.create(
        model="deepseek-r1",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    result = response.choices[0].message.content
    
    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


text = "瑞奇是一个25岁的青年，黑色短发，穿着绿色的夹克。他面无表情地走在乡间小路上。艾米躲在一棵大树后面，眼神中透着不安。"
extract_character_features(text)

NameError: name 're' is not defined

In [30]:
from openai import OpenAI
import re
import os
import json
def extract_character_features(text):
    prompt = f"""
请阅读以下中文小说文本，并识别所有明确提到的“角色名”及其“特征描述”，并将结果用 JSON 结构返回。
JSON的键值对为"角色名": "xxx", "特征": "xxx"
注意：
1. 角色名应当真实在文本中出现。
2. 特征包括年龄、外貌、衣着、行为、情绪等明显描述
3. 如果人物的特征不足甚至没有的话，可以根据文本内容编造额外信息和特征。
例子：
  {{"角色名1": "帕奇",
  "特征1": "一个20岁的黑发男子",
  
  "角色名2": "",
  "特征2": "",
  
  "角色名3": "",
  "特征3": ""}}

小说内容如下：
{text}
"""
    client = OpenAI(
        # 如果没有配置环境变量，请用百炼API Key替换：api_key="sk-xxx"
        api_key = "sk-db3f839bc51e459dae3aab49d1a779e2",
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    
    response = client.chat.completions.create(
        model="qwen-plus",  # 或 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "你是一个擅长文本分析的中文助手。"},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature= 0.5
    )

    result = response.choices[0].message.content
    
    # 尝试解析 JSON
    try:
        # 步骤 1：去除 Markdown 包裹符
        cleaned = re.sub(r'^```json|```$', '', result.strip(), flags=re.MULTILINE).strip()
        # 步骤 2：解析为 Python 对象
        json_data = json.loads(cleaned)
        with open("角色信息.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        return json_data
    except json.JSONDecodeError:
        print("返回内容不是合法的 JSON，可以考虑用正则或人工检查")
        return result


text = '''三只小猪
在一个遥远的山村里，住着一位猪妈妈和她3只可爱的小猪。妈妈每天很辛苦，小猪们一天天长大了，可还是什么事都不做。
一天晚上，吃过晚饭，猪妈妈把孩子们叫到面前郑重其事地说：“你们已经长大了，应该独立生活了，等你们盖好自己的房后就搬出去住吧。“
三只小猪谁也不想搬出去住，更不想自己动手盖房子，又不能不听妈妈的话。于是，他们开始琢磨什么样的房子。老大先动手了。
他首先扛来许多稻草，选择了一片空地，在中间搭了一座简易的稻草屋，然后用草绳捆了捆。“哈哈！我有自己的房子了！”老大乐得欢蹦乱跳。
第二天老大搬进了自己的新家，老二和老三好奇地前来参观。老二说：“老三，你看大哥的房子，也太简陋了，我要盖一座又漂亮、又舒适的房子！“
老二跑到山上砍下许多木头回来，锯成木板、木条，叮叮当当地敲个不停。不久，老二也盖好了自己的木房子。显然这比老大的要漂亮、结实得多。
老二很快搬到自己的新家住了，老大和老三也过来参观。老大赞不绝口，深感自己的房子过于简陋; 老三看后说：“我盖的房子还会更好的。“
老三回到家左思右想，终于决定建造一栋用砖石砌成的房子，因为这种房子非常坚固，不怕风吹雨打，可这需要付出许多努力啊！
老三每天起早贪黑，一趟一趟地搬回一块一块的石头，堆在一旁，再一块一块地砌成一面面墙。哥哥们在一旁取笑道：“只有傻瓜才会这么做！“
小弟毫不理会，仍夜以继日地工作。哥哥们休息了，他还在不停地干。这样整整过了三个月，老三的新房子也盖好了！他好高兴啊！
有一天来了一只大野狼。老大惊慌地躲进了他的稻草屋。野狼"嘿嘿"地冷笑了两声，狼狠吹了口气就把稻草屋吹倒了。老大只好撒腿就跑。
老大径直跑到二弟家，边跑边喊：“二弟！快开门！救命啊！＂二弟打开门一看，一只大野狼追了过来，赶紧让大哥进了屋，关好门。
大野狼追到门前停了下来，心想：“你们以为木头房子就能难住我吗？”他一下一下地向大门撞去。"哗啦”一声，木头房子被撞倒了。
兄弟俩又拼命逃到老三家，气喘吁吁地告诉老三所发生的一切。老三先关紧了门窗，然后胸有成竹地说：“别怕！没问题了！"
大野狼站在大门前，他知道房子里有三只小猪，可不知怎么才能进去。他只能重施旧技，对着房门呼呼吹气，结果无济于事。
野狼有点儿急了，他又用力去撞。“当”的一声，野狼只觉得两眼直冒金星，再看房子，纹丝不动。野狼真的急了，转身去找了一把锤子。
野狼憋足劲，挥起大铁锤敲了下去，没想到锤子把儿断了，锤子反弹回来，正砸在野狼的头上。“疼死我了！”野狼大叫。他真的无技可施了。
野狼只好满脸堆笑地请三只小猪一起去郊游。三只小猪很聪明，也很团结。他们提前到郊外摘了许多苹果。不久，野狼来了。
三只小猪按计划迅速爬到苹果树上。野狼迷惑不解地问：“你们到树上去干什么？＂老三回答说："我们在吃苹果呢！你要不要来一个？“
野狼馋得直流口水，便满口答应了。老三摘了一个大苹果丢下去，苹果顺着山坡滚下好远，野狼在后面追，结果越跑越远。三只小猪趁机跑回了家。
野狼气急败坏地返回来，他绕着房子转了一圈，最后爬上房顶，他想从烟囱溜进去。老三从窗口发现后，马上点起了火。
野狼掉进火炉里，熏得够呛，整条尾巴都烧焦了。他豪叫着夹着尾巴逃走了，再也不敢来找三只小猪的麻烦了。'''
print(extract_character_features(text))

ValueError: Invalid format specifier

In [9]:
import spacy
from spacy.cli import download
import json
from collections import defaultdict

# 自动下载模型
download("zh_core_web_sm")
nlp = spacy.load("zh_core_web_sm")  # 中文模型

def extract_characters_and_traits(text, max_characters=8):
    doc = nlp(text)
    print(doc.ents)
    character_data = defaultdict(list)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            sent = ent.sent
            for token in sent:
                if token.text != name and token.pos_ in ["ADJ", "NOUN", "VERB", "NUM"]:
                    character_data[name].append(token.text)

    # 整理为 JSON 格式
    result = {}
    for idx, (name, traits) in enumerate(character_data.items()):
        if idx >= max_characters:
            break
        suffix = str(idx + 1) if idx > 0 else ""
        result[f"角色名{suffix}"] = name
        result[f"特征{suffix}"] = "，".join(traits)

    # 补空项
    for i in range(len(character_data), max_characters):
        suffix = str(i + 1) if i > 0 else ""
        result[f"角色名{suffix}"] = ""
        result[f"特征{suffix}"] = ""

    return result

# 示例
text = "瑞奇是一个25岁的青年，黑色短发，穿着绿色的夹克。他面无表情地走在乡间小路上。艾米躲在一棵大树后面，眼神中透着不安。"
result = extract_characters_and_traits(text)
print(json.dumps(result, ensure_ascii=False, indent=2))


✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
(25岁, 艾米)
{
  "角色名": "艾米",
  "特征": "躲，一，棵，后面，眼神，透，不安",
  "角色名2": "",
  "特征2": "",
  "角色名3": "",
  "特征3": "",
  "角色名4": "",
  "特征4": "",
  "角色名5": "",
  "特征5": "",
  "角色名6": "",
  "特征6": "",
  "角色名7": "",
  "特征7": "",
  "角色名8": "",
  "特征8": ""
}
